# Mapa

In [ ]:
import folium
import json
from branca.element import Template, MacroElement

# Criar o mapa
m = folium.Map(location=[-15, -55], zoom_start=5,width="100%" ,height="400px")


""" 
Adicionando 1º camada
"""
# Carregar o arquivo GeoJSON
geojson_path1 = 'Bacias_com_Trechos_de_Rios_Estaduais_Criticos.geojson'
with open(geojson_path1, 'r', encoding='utf-8') as f:
    geojson_data1 = json.load(f)

# Adicionar o GeoJSON com campos corrigidos
fg1 = folium.FeatureGroup(name='1ª Camada - Bacias e Rios Críticos', show=True).add_to(m)
folium.GeoJson(
    geojson_data1,
    style_function=lambda feature: {
        'color': '#1f78b4',
        'weight': 2.5,
        'fillOpacity': 0.2,
        'fillColor': '#a6cee3'
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['OBJECTID','BCE_NM_NOME', 'BCE_NM_RIOPRINCIPAL'],
        aliases=['OBJECTID','BCE_NM_NOME:', 'BCE_NM_RIOPRINCIPAL:'],
        localize=True
    )
).add_to(fg1)

""" 
Adicionando 2º camada
"""
# Carregar o arquivo GeoJSON
geojson_path2 = 'Bacia_HidrogrA1fica_Estadual_com_Enquadramento_dos_Corpos_d_81gua_(situaA7A3o_2020).geojson'
with open(geojson_path2, 'r', encoding='utf-8') as f:
    geojson_data2 = json.load(f)

# Adicionar o GeoJSON com campos corrigidos
fg2 = folium.FeatureGroup(name="2ª Camada - Bacias Hid. Est. com Enq. dos Corpos D'agua", show=True).add_to(m)
folium.GeoJson(
    geojson_data2,
    style_function=lambda feature: {
        'color': '#e31a1c',
        'weight': 2,
        'fillOpacity': 0.3,
        'fillColor': '#fb9a99'
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['OBJECTID', 'NULEG'],
        aliases=['OBJECTID:', 'NULEG:'],
        localize=True
    )
).add_to(fg2)

""" 
Adicionando 3º camada
"""
# Carregar o arquivo GeoJSON
geojson_path3 = 'ComitAAs_de_Bacias_HidrogrA1ficas_Interestaduais.geojson'
with open(geojson_path3, 'r', encoding='utf-8') as f:  # Corrigido para geojson_path3
    geojson_data3 = json.load(f)

# Adicionar o GeoJSON com campos corrigidos
fg3 = folium.FeatureGroup(name="3ª Camada - Bacias Hid. Interestaduais", show=True).add_to(m)
folium.GeoJson(
    geojson_data3,
    style_function=lambda feature: {
        'color': "#e3891a",
        'weight': 2,
        'fillOpacity': 0.3,
        'fillColor': "#d3b454"
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["OBJECTID", "POPURB", "POPRUR", "POPTOT", "PIBTOT"],
        aliases=["OBJECTID:", "POPURB:", "POPRUR:", "POPTOT:", "PIBTOT:"],  # Melhorado os aliases
        localize=True
    )
).add_to(fg3)

""" 
Adicionando 4º camada - PRODES (Pontos de Despoluição)
"""
geojson_path4 = 'Programa_de_DespoluiA7A3o_de_Bacias_HidrogrA1ficas_-_PRODES.geojson'
with open(geojson_path4, 'r', encoding='utf-8') as f:
    geojson_data4 = json.load(f)

# Criar grupo para os pontos
fg4 = folium.FeatureGroup(name="4ª Camada - PRODES (Pontos de Despoluição)", show=True).add_to(m)

# Cor rosa vibrante para os pontos
cor_rosa = '#e31ae3'

# Adicionar cada ponto individualmente para melhor controle
for feature in geojson_data4['features']:
    if feature['geometry']['type'] == 'Point':
        # Coordenadas (GeoJSON: [longitude, latitude])
        lon, lat = feature['geometry']['coordinates']
        
        # Popup com informações
        popup_text = f"""
        <h2>Ponto de Despoluição</h2>
        <b>Id:</b> {feature['properties']['OBJECTID']}<br>
        <b>Município:</b> {feature['properties']['MUNICOPIO']}<br>
        <b>UF:</b> {feature['properties']['UF']}<br>
        <b>ETE:</b> {feature['properties']['ETE']}<br>
        <b>Ano:</b> {feature['properties']['ANO']}<br>
        <b>Bacia:</b> {feature['properties']['BACIA']}
        """
        popup = folium.Popup(popup_text, max_width=250)
        
        # Criar marcador circular rosa
        folium.CircleMarker(
            location=[lat, lon],
            radius=6,  # Tamanho do ponto
            color=cor_rosa,
            fill=True,
            fill_color=cor_rosa,
            fill_opacity=0.8,
            weight=1,
            popup=popup
        ).add_to(fg4)

# Adicionar estilo também ao GeoJSON para compatibilidade
folium.GeoJson(
    geojson_data4,
    style_function=lambda feature: {
        'color': cor_rosa,
        'weight': 2,
        'fillOpacity': 0.3,
        'fillColor': cor_rosa
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["OBJECTID", "MUNICOPIO","UF", "ETE", "ANO","BACIA"],
        aliases=["ID:", "Município:","UF", "ETE:", "Ano:","BACIA"],
        localize=True
    )
).add_to(fg4)

# Adicionar controle de camadas
folium.LayerControl(collapsed=False).add_to(m)

# ===== TEMPLATE HTML CORRIGIDO =====
template = """
{% macro html(this, kwargs) %}
<!doctype html>
<html lang="pt">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Mapa de Bacias Hidrográficas</title>
  <style>
        /* Reset do corpo da página */
        html, body {
            width: 100%;
            height: 100%;
            margin: 0;
            padding: 0;
            font-family: 'Arial', sans-serif;
            justify-content: center;
        }
        .container{
          width: 100%;
          height: 100%;
          justify-content: center;
        }

        iframe{
            width: 80%;
            height: 80%;
            justify-content: center;
        }

        /* Container do mapa */
        .map-container {
            display: flex;
            justify-content: center;
            width: 100%;
            margin: 20px 0;
        }

        /* Estilo do mapa Folium */
        #map_9f4ec825d171fbdee8d6708587b83ed1 {
            width: 80%; /* Ajuste conforme necessário */
            height: 60vh; /* Altura responsiva */
            border-radius: 8px;
            box-shadow: 0 2px 6px rgba(0,0,0,0.1);
        }


    </style>
</head>
<body>
  <div class="container">
    <!-- Mapa -->
    <div id="map-container">
      <iframe id="map"></iframe>
    </div>
  </div>
</body>
</html>
{% endmacro %}
"""

macro = MacroElement()
macro._template = Template(template)
m.get_root().add_child(macro)

# Salvar o mapa
m.save('mapa_multicamadas_bacias_hidrograficas.html')

print("Mapa criado/atualizado com sucesso!")

Mapa criado/atualizado com sucesso!
